In [2]:
import os
import pandas as pd
import numpy as np

from utils.loader_steve import load_all_X_y

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# set correct working directory
wd = os.getcwd()
print("Current working directory: ", wd)
if wd[-9:] == "notebooks": wd = wd[:-10]
else: print("Make sure your working directory is set to the student-performance folder!")
os.chdir(wd)
print("New working directory: ", os.getcwd())

data_all, labels = load_all_X_y()

Current working directory:  /Users/nzuchna/Desktop/Drive/2. Areas/University/Master-TUC/M2/4_Forschungsmodul/student-performance/notebooks
New working directory:  /Users/nzuchna/Desktop/Drive/2. Areas/University/Master-TUC/M2/4_Forschungsmodul/student-performance


In [3]:
labels = pd.read_csv("data/processed/labels.csv")
y = labels['correct'][labels['q'] == 1].values
dist = np.unique(y, return_counts=True)[1]  # [0, 1]
print(dist)
prob_0 = dist[0] / (dist[0] + dist[1])
prob_1 = dist[1] / (dist[0] + dist[1])
print(prob_0, prob_1)

[ 6421 17141]
0.2725150666327137 0.7274849333672863


In [7]:
data_all['5_12']

,session_id_1,level_group_1,event_name_1,name_1,fqid_1,room_fqid_1,text_fqid_1,fullscreen_1,hq_1,music_1,...,page_std_8,hover_duration_std_8,difference_clicks_std_8,distance_clicks_std_8,screen_distance_clicks_std_8,index_sum_of_actions_8,difference_clicks_max_8,elapsed_time_max_8,sum_distance_clicks_max_8,clicks_per_second_8
0,20090312431273200,5-12,5,2,6,4,2,0,0,1,...,-1.000000,106.066017,0.617889,334.101776,271.571899,10,1.966,8.089,1720.914062,1.236247
1,20090312433251036,5-12,5,2,8,5,3,0,0,0,...,0.000000,176.776703,1.440249,434.543121,275.379242,10,5.155,18.548,3243.648438,0.539142
2,20090312455206810,5-12,4,2,6,4,1,1,1,1,...,0.000000,62.851677,0.842236,495.198303,451.574158,10,2.215,10.910,3210.085938,0.916590
3,20090313091715820,5-12,5,4,7,5,1,1,1,1,...,-1.000000,-1.000000,1.212265,253.962982,234.349609,10,4.145,12.855,2095.898438,0.777907
4,20090313571836404,5-12,4,2,6,4,1,0,0,1,...,-1.000000,-1.000000,0.619625,205.166504,229.803146,8,1.972,9.890,2142.171875,0.808898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23557,22100215342220508,5-12,4,3,7,5,2,1,0,1,...,-1.000000,60.324123,2.230104,328.046783,271.191864,9,6.975,14.920,1761.117188,0.603217
23558,22100215460321130,5-12,7,3,10,5,2,0,0,1,...,-1.000000,-1.000000,1.328251,343.401031,290.064606,6,3.980,11.268,1665.062500,0.532481
23559,22100217104993650,5-12,4,3,7,4,1,0,0,1,...,0.000000,-1.000000,0.678011,391.206177,235.494781,29,2.750,28.268,10165.039062,1.025895
23560,22100219442786200,5-12,4,2,6,4,1,0,0,1,...,1.100505,248.901581,0.953770,347.272583,321.641907,18,3.474,19.150,3709.859375,0.939948


In [8]:
# labels = pd.read_csv("data/processed/labels.csv")
models = {}

for q in labels['q'].unique():
    # get X
    X = data_all['5_12'].iloc[:, 2:].values
    # get y
    y = labels['correct'][labels['q'] == q].values
    # Creating training and test split
    _, _, _, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
    # get distribution of y
    dist = np.unique(y_test, return_counts=True)[1]
    prob_0 = dist[0] / (dist[0] + dist[1])
    if prob_0 > 0.5:
        y_pred = np.zeros(len(y_test))
    else:
        y_pred = np.ones(len(y_test))

    # Get accuracy
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Trained model for question {q}, accuracy: {round(acc, 2)}, f1_score: {round(f1, 2)}")
    models[f"Question_{q}"] = {'model' : None, 'prob_0': prob_0, 'prob_1': prob_1, 'acc': acc, 'f1_score': f1}

print(f"Mean accuracy: {round(np.mean([models[q]['acc'] for q in models.keys()]), 4)}")
print(f"Mean f1_score: {round(np.mean([models[q]['f1_score'] for q in models.keys()]), 4)}")

Trained model for question 1, accuracy: 0.73, f1_score: 0.84
Trained model for question 2, accuracy: 0.98, f1_score: 0.99
Trained model for question 3, accuracy: 0.93, f1_score: 0.97
Trained model for question 4, accuracy: 0.8, f1_score: 0.89
Trained model for question 5, accuracy: 0.55, f1_score: 0.71
Trained model for question 6, accuracy: 0.78, f1_score: 0.87
Trained model for question 7, accuracy: 0.74, f1_score: 0.85
Trained model for question 8, accuracy: 0.62, f1_score: 0.76
Trained model for question 9, accuracy: 0.74, f1_score: 0.85
Trained model for question 10, accuracy: 0.51, f1_score: 0.67
Trained model for question 11, accuracy: 0.64, f1_score: 0.78
Trained model for question 12, accuracy: 0.86, f1_score: 0.93
Trained model for question 13, accuracy: 0.72, f1_score: 0.0
Trained model for question 14, accuracy: 0.71, f1_score: 0.83
Trained model for question 15, accuracy: 0.52, f1_score: 0.0
Trained model for question 16, accuracy: 0.73, f1_score: 0.85
Trained model for qu

In [14]:
# labels = pd.read_csv("data/processed/labels.csv")
models = {}

for q in labels['q'].unique():
    # get X
    X = data_all['13_22'].iloc[:, 2:].values
    # get y
    y = labels['correct'][labels['q'] == q].values
    # Creating training and test split
    _, _, _, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
    # get distribution of y
    dist = np.unique(y_test, return_counts=True)[1]
    prob_0 = dist[0] / (dist[0] + dist[1])
    if prob_0 > 0.5:
        y_pred = np.zeros(len(y_test))
    else:
        y_pred = np.ones(len(y_test))

    # Get accuracy
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Trained model for question {q}, accuracy: {round(acc, 2)}, f1_score: {round(f1, 2)}")
    models[f"{q}"] = {'model' : None, 'prob_0': prob_0, 'prob_1': prob_1, 'acc': acc, 'f1_score': f1}

print(f"Mean accuracy: {round(np.mean([models[q]['acc'] for q in models.keys()]), 4)}")
print(f"Mean f1_score: {round(np.mean([models[q]['f1_score'] for q in models.keys()]), 4)}")

Trained model for question 1, accuracy: 0.73, f1_score: 0.84
Trained model for question 2, accuracy: 0.98, f1_score: 0.99
Trained model for question 3, accuracy: 0.93, f1_score: 0.97
Trained model for question 4, accuracy: 0.8, f1_score: 0.89
Trained model for question 5, accuracy: 0.55, f1_score: 0.71
Trained model for question 6, accuracy: 0.78, f1_score: 0.87
Trained model for question 7, accuracy: 0.74, f1_score: 0.85
Trained model for question 8, accuracy: 0.62, f1_score: 0.76
Trained model for question 9, accuracy: 0.74, f1_score: 0.85
Trained model for question 10, accuracy: 0.51, f1_score: 0.67
Trained model for question 11, accuracy: 0.64, f1_score: 0.78
Trained model for question 12, accuracy: 0.86, f1_score: 0.93
Trained model for question 13, accuracy: 0.72, f1_score: 0.0
Trained model for question 14, accuracy: 0.71, f1_score: 0.83
Trained model for question 15, accuracy: 0.52, f1_score: 0.0
Trained model for question 16, accuracy: 0.73, f1_score: 0.85
Trained model for qu

In [15]:
models

{'1': {'model': None,
  'prob_0': 0.2724572075258169,
  'prob_1': 0.7274849333672863,
  'acc': 0.7275427924741831,
  'f1_score': 0.8422862757943007},
 '2': {'model': None,
  'prob_0': 0.02121940868581129,
  'prob_1': 0.7274849333672863,
  'acc': 0.9787805913141887,
  'f1_score': 0.9892765227337718},
 '3': {'model': None,
  'prob_0': 0.06606309237515914,
  'prob_1': 0.7274849333672863,
  'acc': 0.9339369076248408,
  'f1_score': 0.9658400994806525},
 '4': {'model': None,
  'prob_0': 0.20172584523977932,
  'prob_1': 0.7274849333672863,
  'acc': 0.7982741547602207,
  'f1_score': 0.8878225298930145},
 '5': {'model': None,
  'prob_0': 0.4516904795586363,
  'prob_1': 0.7274849333672863,
  'acc': 0.5483095204413637,
  'f1_score': 0.7082686158063043},
 '6': {'model': None,
  'prob_0': 0.2240769557221672,
  'prob_1': 0.7274849333672863,
  'acc': 0.7759230442778328,
  'f1_score': 0.8738250756730923},
 '7': {'model': None,
  'prob_0': 0.2639694440514924,
  'prob_1': 0.7274849333672863,
  'acc': 0.

In [12]:
# Reference
# https://www.kaggle.com/code/philculliton/basic-submission-demo
# https://www.kaggle.com/code/cdeotte/random-forest-baseline-0-664/notebook

from data.raw import jo_wilder
from utils.data_preprocessing_steve import pp_pipeline_noah
env = jo_wilder.make_env()
iter_test = env.iter_test()

limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}

for (test, sample_submission) in iter_test:
    test_df = pp_pipeline_noah(data=test, file_path=None, flatten=True, saveIntermediateFiles=False, dtypes=None, output=True)
    grp = test_df.level_group.values[0]
    a,b = limits[grp]
    for t in range(a,b):
        nbc = models[f"Question_{t}"]
        if nbc['prob_0'] > 0.5:
            predictions = np.zeros(len(test_df))
            best_threshold = nbc['prob_0']
        else:
            predictions = np.ones(len(test_df))
            best_threshold = nbc['prob_1']
        mask = sample_submission.session_id.str.contains(f'q{t}')
        n_predictions = (predictions > best_threshold).astype(int)
        sample_submission.loc[mask,'correct'] = n_predictions.flatten()

    env.predict(sample_submission)

ModuleNotFoundError: No module named 'data.raw.jo_wilder.competition'